In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import datetime
import re

In [ ]:
path = str(Path(os.path.abspath(os.getcwd())).absolute())  #my directory
dataPath = path + "\plsample.csv" 
countriesURL = path + '\wikipedia-iso-country-codes-edit.csv'
dtypes = {'Name':str, 
          'Sex': str,
          'Event':str,
          'Equipment':str, 
          'Age': np.float32,
          'AgeClass':str,
          'Division':str,
          'BodyweightKg':np.float32,
          'WeightClassKg':str, 
          'Squat1Kg':np.float32,
          'Squat2Kg':np.float32,
          'Squat3Kg':np.float32,
          'Squat4Kg':np.float32,
          'Best3SquatKg':np.float32,
          'Bench1Kg':np.float32,
          'Bench2Kg':np.float32,
          'Bench3Kg':np.float32,
          'Bench4Kg':np.float32,
          'Best3BenchKg':np.float32,
          'Deadlift1Kg':np.float32,
          'Deadlift2Kg':np.float32,
          'Deadlift3Kg':np.float32,
          'Deadlift4Kg':np.float32,
          'Best3DeadliftKg':np.float32,
          'TotalKg':np.float32,
          'Place':str, #DQ=-1,G=-2,NS=-3,DD=-4
          'Wilks':np.float32,
          'McCulloch':np.float32,
          'Glossbrenner':np.float32,
          'IPFPoints':np.float32,
          'Tested': str,
          'Country':str,
          'Federation':str,
          'Date':object, 
          'MeetCountry':str,
          'MeetState':str,
          'MeetName':str
         }
pldata = pd.read_csv(dataPath, dtype=dtypes, parse_dates=['Date'], infer_datetime_format = True)
countries = pd.read_csv(countriesURL, sep=',', index_col='Name', keep_default_na=False, na_values=['_'])

In [ ]:
pldata.info()

In [ ]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
PL = Namespace("http://www.semanticweb.org/mario/ontologies/2022/10/powerlifting#")

g = Graph()

g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("pl", PL)

In [ ]:
%%time
print('Starting graph building.....')
namae = ''
indox = 0
nRows = len(pldata.index)
print('Total rows number: ' + str(nRows))
try:
    for index, row in pldata.iterrows():
        #percentage of completion print
        indox = index
        if indox % (nRows / 100) <= 1:
            print(str(round(indox / (nRows/100))),'%')
        #add Meeting and Federation
        date = row['Date']
        mName = row['MeetName'].replace(" ","_")
        compMName = re.sub('[^A-Za-z0-9_]', '', mName) + row['MeetCountry'].replace(' ','') + str(date.year) 
        Meeting = URIRef(PL[compMName])
        fedName = row['Federation'].strip()
        Federation = URIRef(PL[fedName])
        g.add((Meeting, RDF.type, PL.Meeting))
        g.add((Federation, RDF.type, PL.Federation))
        g.add((Federation, PL['name'], Literal(fedName, datatype = XSD.string)))
        g.add((Meeting, PL['hostedBy'], Federation))
        g.add((Meeting, PL['name'], Literal(mName, datatype = XSD.string)))
        g.add((Meeting, PL['year'], Literal(date.year, datatype = XSD.int))) #optional
        
        
        
        #add people
        pName = row['Name'].split(' ') 
        fName = ''
        offN = 1
        #omonyms
        if (pName[-1]).startswith('#'):
            offN = 2
        lName = pName[-offN]
        if len(pName) > offN:
            fName = pName[0]
            for i in pName[1:-offN]:
                lName = (i + ' ' + lName).strip()    
        compPName = row['Name'].lower().replace(" ","").replace('#','')
        Person = URIRef(PL[compPName])
        g.add((Person, RDF.type, PL.Athlete))
        namae = fName + lName
        g.add((Person, FOAF['name'], Literal(fName + ' ' + lName, datatype = XSD.string)))
        #model first-last name, some have just last name
        g.add((Person, FOAF['surname'], Literal(lName, datatype = XSD.string)))
        if (fName != ''):
            g.add((Person, FOAF['firstName'], Literal(fName, datatype = XSD.string)))
        if (row['Sex'] == 'M'):
            g.add((Person, PL['sex'], Literal(True, datatype = XSD.boolean)))
        else:
            g.add((Person, PL['sex'], Literal(False, datatype = XSD.boolean)))
            
            
            
        #manage athlete countries    
        if not pd.isna(row['Country']):
            cName = row['Country'].strip()
            if((countries.index == cName).any() == True):             
                code = str(countries[countries.index == cName]['Alpha-2 code'][0]).lower()               
            elif cName == 'USA':
                code = 'us'
                cName = 'United States'
            #english states
            elif cName == 'UK' or cName == 'England' or cName == 'Scotland' or cName == 'Wales' or cName == 'N.Ireland':
                code = 'gb' 
                cName = 'United Kingdom'
            #URSS assieme a russia
            elif cName == 'Russia' or cName == 'USSR': 
                code = 'ru'  
                cName = 'Russian Federation'
            #emirati arabi
            elif cName == 'UAE': 
                code = 'ae' 
                cName = 'United Arab Emirates'
            #transnistria
            elif cName == 'Transnistria': 
                code = 'md'
                cName = 'Moldova'
            #tahiti
            elif cName == 'Tahiti':
                code = 'pf' 
                cName = 'French Polynesia'
            #berlin wall
            elif cName == 'West Germany' or cName == 'East Germay':
                code = 'de'
                cName = 'Germany'
            #cape verde
            elif cName == 'Cabo Verde':
                code = 'cv'
                cName = 'Cape Verde'
            #timor
            elif cName == 'East Timor':
                code = 'tl'     
                cName = 'Timor-Leste'  
            #serbia and montenegro
            elif cName == 'Serbia and Montenegro':
                code = 'rs' 
                cName = 'Serbia'
            #not recognized
            else:
                code = row['Country'].replace(' ','').replace('#','').lower()
                cName = code
                print(index, row['Country'])
            Country = URIRef(CNS[code])
            g.add((Country, RDF.type, PL.Country))
            g.add((Country, PL['name'], Literal(cName, datatype = XSD.string)))
            g.add((Person, PL['nationality'], Country))
        
        
        #meeting country
        MeetState = None
        if not pd.isna(row['MeetCountry']):
            mcName = row['MeetCountry'].strip()
            if((countries.index == mcName).any() == True):             
                mcode = str(countries[countries.index == mcName]['Alpha-2 code'][0]).lower()                   
            #USA
            elif mcName == 'USA':
                mcode = 'us'
                mcName = 'United States'
            #english states
            elif mcName == 'UK' or mcName == 'England' or mcName == 'Scotland' or mcName == 'Wales' or mcName == 'N.Ireland':
                mcode = 'gb'                
                if mcName == 'England':
                    MeetState = URIRef(PL[mcode + '_ENG'])
                elif mcName == 'Scotland':
                    MeetState = URIRef(PL[mcode + '_SCO'])
                elif mcName == 'Wales':
                    MeetState = URIRef(PL[mcode + '_WAL'])
                elif mcName == 'N.Ireland':
                    MeetState = URIRef(PL[mcode + '_NIR'])  
                mcName = 'United Kingdom'
            #URSS assieme a russia
            elif mcName == 'Russia' or mcName == 'USSR': 
                mcode = 'ru'   
                mcName = 'Russian federation'
            #emirati arabi
            elif mcName == 'UAE': 
                mcode = 'ae'  
                mcName = 'United Arab Emirates'
            #timor
            elif mcName == 'East Timor':
                mcode = 'tl' 
                mcName = 'Timor-Leste'
            #serbia and montenegro
            elif mcName == 'Serbia and Montenegro':
                mcode = 'rs' 
                mcName = 'Serbia'
            #cape verde
            elif mcName == 'Cabo Verde':
                mcode = 'cv'
                mcName = 'Cape Verde'
            #berlin wall
            elif mcName == 'West Germany' or mcName == 'East Germay':
                mcode = 'de'
                mcName = 'Germany'
            #tahiti
            elif mcName == 'Tahiti':
                mcode = 'pf'     
                mcName = 'French Polynesia'
                MeetState = URIRef(PL[mcode + '_TAH'])  
            #transnistria
            elif mcName == 'Transnistria': 
                mcode = 'md'
                mcName = 'Moldova'
                MeetState = URIRef(PL[mcode + '_TNS'])
            #not recognized
            else:
                mcode = row['MeetCountry'].replace(' ','').replace('#','').lower()
                mcName = mcode
                print(row['MeetCountry'])
            MeetCountry = URIRef(CNS[mcode])
            g.add((MeetCountry, RDF.type, PL.Country))
            g.add((MeetCountry, PL['name'], Literal(mcName, datatype = XSD.string)))
            g.add((Meeting, PL['nationality'], MeetCountry))
            #can add state
            if MeetState != None:
                g.add((MeetState, RDF.type, PL.MeetState))                
                g.add((MeetCountry, PL['hasState'], MeetState))
                g.add((Meeting, PL['inState'], MeetState))
            elif not pd.isna(row['MeetState']):
                state = row['MeetState'].replace(' ','').replace('#','')
                MeetState = URIRef(PL[mcode + '_' + state])
                g.add((MeetState, RDF.type, PL.MeetState))
                g.add((MeetCountry, PL['hasState'], MeetState))
                g.add((Meeting, PL['inState'], MeetState))
                
                
                
        #add competition
        compId = compPName + '_' + compMName + '_' + row['Event']
        Competition = URIRef(PL[compId])
        g.add((Competition, RDF.type, PL.Competition))
        g.add((Person, PL['did'], Competition))
        g.add((Competition, PL['doneDuring'], Meeting))
        g.add((Competition, PL['inEvent'], Literal(row['Event'].strip(), datatype = XSD.string))) 
        g.add((Competition, PL['date'], Literal(date, datatype = XSD.date)))
        if (row['Equipment'] == 'Wraps' or row['Equipment'] == 'Raw'):
            equip = 'R'
        elif row['Equipment'] == 'Single-ply' or row['Equipment'] == 'Multi-ply' or row['Equipment'] == 'Straps':
            equip = 'E'
        else:
            equip = row['Equipment']
            print(index, equip)
        Equipment = URIRef(PL[row['Equipment'].strip().capitalize()])
        g.add((Equipment, RDF.type, PL.Equipment))
        g.add((Competition, PL['equipment'], Equipment))  
        age = -1
        if not pd.isna(row['Age']):
            age = round(row['Age'])
            if age > 0:
                g.add((Competition, PL['athleteAge'], Literal(age, datatype = XSD.int)))
                if not (Person, PL['birthYear'], None) in g:
                    g.add((Person, PL['birthYear'], Literal(date.year - age, datatype = XSD.int)))
        hasADiv = False        
        if not pd.isna(row['AgeClass']):
            if (age >= 0) or (not any(c.isalpha() for c in row['AgeClass'])):
                hasADiv = True
                g.add((Competition, PL['ageDivision'], Literal(row['AgeClass'], datatype = XSD.string)))        
        if not pd.isna(row['Division']):
            g.add((Competition, PL['meetDivision'], Literal(row['Division'], datatype = XSD.string)))
        weight = 0
        if not pd.isna(row['BodyweightKg']):
            weight = float(row['BodyweightKg'])
            g.add((Competition, PL['athleteWeight'], Literal(weight, datatype = XSD.float)))
        hasWDiv = False
        if not pd.isna(row['WeightClassKg']):
            hasWDiv = True
            g.add((Competition, PL['meetWeightDivision'], Literal(row['WeightClassKg'], datatype = XSD.string)))
        #SQUAT
        if not pd.isna(row['Squat1Kg']):
            g.add((Competition, PL['1stSquat'], Literal(row['Squat1Kg'], datatype = XSD.float)))
        if not pd.isna(row['Squat2Kg']):        
            g.add((Competition, PL['2ndSquat'], Literal(row['Squat2Kg'], datatype = XSD.float)))
        if not pd.isna(row['Squat3Kg']):
            g.add((Competition, PL['3rdSquat'], Literal(row['Squat3Kg'], datatype = XSD.float)))
        if not pd.isna(row['Squat4Kg']):
            g.add((Competition, PL['4thSquat'], Literal(row['Squat4Kg'], datatype = XSD.float)))    
        if not pd.isna(row['Best3SquatKg']):
            if (row['Best3SquatKg'] < 0):
                g.add((Competition, PL['bestSquat'], Literal(0.0, datatype = XSD.float)))
            else:
                g.add((Competition, PL['bestSquat'], Literal(row['Best3SquatKg'], datatype = XSD.float)))
        #BENCH
        if not pd.isna(row['Bench1Kg']):
            g.add((Competition, PL['1stBench'], Literal(row['Bench1Kg'], datatype = XSD.float)))
        if not pd.isna(row['Bench2Kg']):        
            g.add((Competition, PL['2ndBench'], Literal(row['Bench2Kg'], datatype = XSD.float)))
        if not pd.isna(row['Bench3Kg']):
            g.add((Competition, PL['3rdBench'], Literal(row['Bench3Kg'], datatype = XSD.float)))
        if not pd.isna(row['Bench4Kg']):
            g.add((Competition, PL['4thBench'], Literal(row['Bench4Kg'], datatype = XSD.float)))    
        if not pd.isna(row['Best3BenchKg']):
            if (row['Best3BenchKg'] < 0):
                g.add((Competition, PL['bestBench'], Literal(0.0, datatype = XSD.float)))
            else:
                g.add((Competition, PL['bestBench'], Literal(row['Best3BenchKg'], datatype = XSD.float)))
        #DEADLIFT
        if not pd.isna(row['Deadlift1Kg']):
            g.add((Competition, PL['1stDeadlift'], Literal(row['Deadlift1Kg'], datatype = XSD.float)))
        if not pd.isna(row['Deadlift2Kg']):        
            g.add((Competition, PL['2ndDeadlift'], Literal(row['Deadlift2Kg'], datatype = XSD.float)))
        if not pd.isna(row['Deadlift3Kg']):
            g.add((Competition, PL['3rdDeadlift'], Literal(row['Deadlift3Kg'], datatype = XSD.float)))
        if not pd.isna(row['Deadlift4Kg']):
            g.add((Competition, PL['4thDeadlift'], Literal(row['Deadlift4Kg'], datatype = XSD.float)))    
        if not pd.isna(row['Best3DeadliftKg']):
            if (row['Best3DeadliftKg'] < 0):
                g.add((Competition, PL['bestDeadlift'], Literal(0.0, datatype = XSD.float)))
            else:
                g.add((Competition, PL['bestDeadlift'], Literal(row['Best3DeadliftKg'], datatype = XSD.float)))
        #TOTAL RESULTS 
        if not pd.isna(row['TotalKg']):
            g.add((Competition, PL['totalKg'], Literal(row['TotalKg'], datatype = XSD.float)))
        if (row['Place'].isnumeric()):        #CAN'T BE NULL
            g.add((Competition, PL['placed'], Literal(int(row['Place']), datatype = XSD.int))) 
        else:
            if (row['Place'] == 'DQ'): #desqualified
                g.add((Competition, PL['placed'], Literal(-1, datatype = XSD.int)))
            if (row['Place'] == 'G'): #guest athlete
                g.add((Competition, PL['placed'], Literal(-2, datatype = XSD.int)))
            if (row['Place'] == 'NS'): #didn't show
                g.add((Competition, PL['placed'], Literal(-3, datatype = XSD.int)))
            if (row['Place'] == 'DD'): #desqualified for doping
                g.add((Competition, PL['placed'], Literal(-4, datatype = XSD.int)))
        if not pd.isna(row['Wilks']):
            g.add((Competition, PL['wilksScore'], Literal(row['Wilks'], datatype = XSD.float)))
        if not pd.isna(row['McCulloch']):
            g.add((Competition, PL['mcCullochScore'], Literal(row['McCulloch'], datatype = XSD.float)))
        if not pd.isna(row['Glossbrenner']):
            g.add((Competition, PL['glossbrennerScore'], Literal(row['Glossbrenner'], datatype = XSD.float)))
        if not pd.isna(row['IPFPoints']):
            g.add((Competition, PL['IPFScore'], Literal(row['IPFPoints'], datatype = XSD.float)))
        if row['Place'] == -4:
            g.add((Competition, PL['hasBeenDrugTested'], Literal(True, datatype = XSD.boolean)))
        elif not pd.isna(row['Tested']):
            if row['Tested'] == 'Yes':
                g.add((Competition, PL['hasBeenDrugTested'], Literal(True, datatype = XSD.boolean)))
            else:
                g.add((Competition, PL['hasBeenDrugTested'], Literal(False, datatype = XSD.boolean)))
        
        
        #sex division
        cat = row['Sex'].strip()       
        
        
        #age divisions
        if age <= 0 and hasADiv:
            try:
                age = int(row['AgeClass'].strip().split('-')[-1])
            except ValueError:
                print(index, 'AgeClass: ', row['AgeClass'])
            if (age <= 10):
                print(index, pName, age, ' from division')
        elif age <= 0 and (Person, PL['birthYear'], None) in g:
            bYear = int(g.value(subject = Person, predicate = PL['birthYear'], any = False))        
            age = int(date.year) - bYear
            #if (age <= 10) and age != -1:
                #print(index, pName, age, ' from year')
        if age <= 12:
            cat = cat + '_NA' #no age and no way to retrieve put to open by default 
            IPFAgeCategory = URIRef(PL['NotAvailable'])
        elif age < 19:
            cat = cat + '_SJ'
            IPFAgeCategory = URIRef(PL['Sub-Junior'])
        elif age < 24:
            cat = cat + '_JJ' 
            IPFAgeCategory = URIRef(PL['Junior'])
        elif age < 40:
            cat = cat + '_OO' 
            IPFAgeCategory = URIRef(PL['Open'])
        elif age < 50:
            cat = cat + '_M1' 
            IPFAgeCategory = URIRef(PL['Master1'])
        elif age < 60:
            cat = cat + '_M2' 
            IPFAgeCategory = URIRef(PL['Master2'])
        elif age < 70:
            cat = cat + '_M3' 
            IPFAgeCategory = URIRef(PL['Master3'])
        else:
            cat = cat + '_M4' 
            IPFAgeCategory = URIRef(PL['Master4'])
        g.add((IPFAgeCategory, RDF.type, PL.IPFAgeCategory))
        g.add((Competition, PL['IPFAgeClass'], IPFAgeCategory))
        
        
        #weight divisions
        if weight <= 10 and hasWDiv:
            if row['WeightClassKg'].isnumeric():
                weight = int(row['WeightClassKg'])
            else:
                wString = row['WeightClassKg'][:-1]              
                try:
                    weight = float(wString)
                except ValueError:
                    print(wString)
        if weight <= 10:
            cat = cat + '_NULL' #categoria peso non disponibile
            IPFWeightCategory = URIRef(PL['NotAvailable'])

        else:
            if cat[0] == 'M':                               #  male weight
                if weight <= 53 and cat[-2:] == 'SJ': #only for sub-junior
                    cat = cat + '_U053'
                    IPFWeightCategory = URIRef(PL['Under53Kg'])
                elif weight <= 59:
                    cat = cat + '_U059'
                    IPFWeightCategory = URIRef(PL['Under59Kg'])
                elif weight <= 66:
                    cat = cat + '_U066'
                    IPFWeightCategory = URIRef(PL['Under66Kg'])            
                elif weight <= 74:
                    cat = cat + '_U074'
                    IPFWeightCategory = URIRef(PL['Under74Kg'])
                elif weight <= 83:
                    cat = cat + '_U083'
                    IPFWeightCategory = URIRef(PL['Under83Kg'])
                elif weight <= 93:
                    cat = cat + '_U093'
                    IPFWeightCategory = URIRef(PL['Under93Kg'])
                elif weight <= 105:
                    cat = cat + '_U105'
                    IPFWeightCategory = URIRef(PL['Under105Kg'])
                elif weight <= 120:
                    cat = cat + '_U120'
                    IPFWeightCategory = URIRef(PL['Under120Kg'])
                else:
                    cat = cat + '_O120'
                    IPFWeightCategory = URIRef(PL['Over120Kg'])
            else:                                         #  female weight
                if weight <= 43 and cat[-2:] == 'SJ': #only for sub-junior
                    cat = cat + '_U043'
                elif weight <= 47:
                    cat = cat + '_U047'
                    IPFWeightCategory = URIRef(PL['Under47'])
                elif weight <= 52:
                    cat = cat + '_U052'
                    IPFWeightCategory = URIRef(PL['Under52Kg'])            
                elif weight <= 57:
                    cat = cat + '_U057'
                    IPFWeightCategory = URIRef(PL['Under57Kg'])
                elif weight <= 63:
                    cat = cat + '_U063'
                    IPFWeightCategory = URIRef(PL['Under63Kg'])
                elif weight <= 69:
                    cat = cat + '_U069'
                    IPFWeightCategory = URIRef(PL['Under69Kg'])
                elif weight <= 76:
                    cat = cat + '_U076'
                    IPFWeightCategory = URIRef(PL['Under76Kg'])
                elif weight <= 84:
                    cat = cat + '_U084'
                    IPFWeightCategory = URIRef(PL['Under84Kg'])
                else:
                    cat = cat + '_O084'
                    IPFWeightCategory = URIRef(PL['Over84Kg'])
        g.add((IPFWeightCategory, RDF.type, PL.IPFWeightCategory))
        g.add((Competition, PL['IPFWeightClass'], IPFWeightCategory))
        
        #equipment and general category
        cat = cat + '_' + equip
        if len(cat) != 11:
            print(index, cat)
        IPFCategory = URIRef(PL[cat])
        g.add((IPFCategory, RDF.type, PL.IPFCategory))
        g.add((Competition, PL['IPFClass'], IPFCategory))
        g.add((IPFCategory, PL['weightClass'], IPFWeightCategory))
        g.add((IPFCategory, PL['ageClass'], IPFAgeCategory))
        g.add((IPFCategory, PL['allowsUseOf'], Equipment))
        if (row['Sex'] == 'M'):
            g.add((IPFCategory, PL['sex'], Literal(True, datatype = XSD.boolean)))
        else:
            g.add((IPFCategory, PL['sex'], Literal(False, datatype = XSD.boolean)))
except ValueError:
    print(namae, indox, 'ValueError')
except AssertionError:
    print(namae, indox, 'AssertionError')


In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(path + '\pl_db2_serialization.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))